In [135]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

In [139]:
df_fesx = pd.read_csv("./data/fesx_M15_2020-06-20_F.US.DSXM20.scid_BarData.txt", sep=", ", engine = "python")
df_fesx["Datetime"] = pd.to_datetime(df_fesx.Date + "-" + df_fesx.Time,format="%Y/%m/%d-%H:%M:%S")
df_fesx = df_fesx[['Datetime','Open', 'High', 'Low', 'Last', 'Volume']].rename(columns={"Last":"Close"})#.set_index("Datetime")
df_fesx = df_fesx.rename(columns={"Last":"Close"})
df_fesx.head()

,Datetime,Open,High,Low,Close,Volume
0,2020-05-21 02:15:00,2917,2921,2916,2918,837
1,2020-05-21 02:30:00,2917,2921,2916,2920,354
2,2020-05-21 02:45:00,2921,2922,2920,2920,213
3,2020-05-21 03:00:00,2920,2920,2915,2919,293
4,2020-05-21 03:15:00,2919,2919,2916,2917,511


In [180]:
rolling_n = 2
min_diff = 0


diff_high = df_fesx.High.diff().reset_index()
diff_low = df_fesx.Low.diff()
diff_low.index = diff_low.index -(rolling_n)
diff_low = diff_low.reset_index()
diff_df = pd.merge(diff_high, diff_low, on="index", how="left").rename(columns={"High":"diff_high", "Low":"diff_low"})


# For every n elements: high(idx - n)> ... >high(idx - 1)>high(idx - 0) => diff(idx) < 0
# n consecutive elements with decreasing high: rolling(n).max() < 0
diff_df['high_max'] = diff_df.diff_high.rolling(rolling_n).max()
diff_df['low_min'] = diff_df.diff_low.rolling(rolling_n).min()

diff_df['Open'] = False

diff_df.loc[(diff_df.high_max < min_diff) & (diff_df.low_min > min_diff), 'Open'] = True


fig = go.Figure(data= [
    go.Scatter(x=df_fesx.index, y=df_fesx.High,name = 'High',mode="lines+markers", marker_color = "green"),
    go.Scatter(x=df_fesx.index, y=df_fesx.Low,name = 'Low',mode="lines+markers", marker_color = "red"),
    go.Scatter(x=df_fesx[diff_df.Open].index, y=df_fesx[diff_df.Open].High,name = 'Open',mode="markers", marker_color = "orange", marker_size = 2 ),
    
    go.Candlestick(x=df_fesx.index,
                open=df_fesx['Open'],
                high=df_fesx['High'],
                low=df_fesx['Low'],
                close=df_fesx['Close'],
                name = 'Candlestick'
                )
    ]
    )

fig.update_layout(
    xaxis_rangeslider_visible= False
)



for idx, row in df_fesx[diff_df.Open].iterrows():
   exact_point = False
   high = row['High']
   #print(idx, high)    
   # Check if the next point after a open is the same value => Dont consider next point as a touch point
   if  high == df_fesx.loc[idx+1,'High']:
      continue
   # Check if the next point after a open is lower than the open
   elif high > df_fesx.loc[idx+1,'High']:
      df_touch_line =  df_fesx[(df_fesx.index > idx) & (df_fesx.High >= high)]
      if df_touch_line.shape[0] == 0:
         continue
      if df_touch_line.High.iloc[0] == high:
         exact_point = True
   # Check if the next point after a open is higher than the open
   elif high < df_fesx.loc[idx+1,'High']:     
      df_touch_line =  df_fesx[(df_fesx.index > idx) & (df_fesx.High <= high)]
      if df_touch_line.shape[0] == 0:
         continue
      if df_touch_line.High.iloc[0] == high:
         exact_point = True
   
   if not exact_point:
      # Linear interpolation
      y = df_touch_line.High.iloc[0]
      x = df_touch_line.iloc[0].name
      xo = x -1
      yo = df_fesx.loc[xo,'High']
      m = (y - yo)/(x - xo)
      dy = high - yo
      dx = dy/m
      next_idx = xo + dx
   else:
      next_idx = df_touch_line.iloc[0].name


   fig.add_trace(
      go.Scatter(x=[idx, next_idx], y=[high, high],name = 'level',mode="lines", marker_color = "red", showlegend=False)
   )
          

fig.show()

# Check 1060

In [138]:
# Detect when the price goes up 5 consecutive times by at least 2 
df_fesx['Diff'] = df_fesx.High.diff()

rolling_n = 3
min_diff = 0

df_fesx['DiffMin'] = df_fesx.Close.diff().rolling(rolling_n).min()
df_fesx['DiffMax'] = df_fesx.Close.diff().rolling(rolling_n).max()

df_fesx['Buy'] = False
df_fesx['Sell'] = False

mask_buy = (df_fesx.DiffMin> min_diff)
mask_sell = (df_fesx.DiffMax < -min_diff)

df_fesx.loc[mask_buy, 'Buy'] = True
df_fesx.loc[mask_sell, 'Sell'] = True

go.Figure(data= [
    go.Scatter(x=np.arange(df_fesx.shape[0]), y=df_fesx.High,name = 'fesx',mode="lines+markers"),
    go.Scatter(x=np.arange(df_fesx.shape[0]), y=df_fesx.High.rolling(30).mean(), name=f'RM({30})'),
    #go.Scatter(x=df_fesx[mask_buy].index, y=df_fesx[mask_buy].High, mode='markers', marker_color='red', name = 'Buy'),
    #go.Scatter(x=df_fesx[mask_sell].index, y=df_fesx[mask_sell].High, mode='markers', marker_color='green', name= 'Sell'),

    go.Candlestick(x=np.arange(df_fesx.shape[0]),
                open=df_fesx['Open'],
                high=df_fesx['High'],
                low=df_fesx['Low'],
                close=df_fesx['Close'])
    ]            
    )



In [ ]:
def consec_n_up(arr:pd.Series,n, min_diff):
    arr = pd.Series(arr)
    return arr.iloc[-(n+1):].diff().min()>min_diff

def consec_n_down(arr:pd.Series,n, min_diff):
    arr = pd.Series(arr)
    return arr.iloc[-(n+1):].diff().max()<-min_diff

def fesx(arr : pd.Series):
    return arr

In [ ]:
ns =np.arange(1,11)
min_diffs = np.linspace(0,5,10)
profit = np.zeros((ns.shape[0], min_diffs.shape[0]))

In [ ]:
for n_idx, n in enumerate(ns):
    for min_diff_idx, min_diff in enumerate(min_diffs):
        class Teste1(Strategy):
                def init(self):
                    pass
                #    high = self.data.High
                #    self.array = self.I(fesx, high,scatter=False)
                def next(self):
                    if consec_n_down(self.data.High, n, min_diff):
                        self.sell()
                    if consec_n_up(self.data.High, n, min_diff):
                        self.buy()


        bt = Backtest(df_fesx, Teste1,
                    cash=10000, commission=.002,
                    exclusive_orders=True)

        output = bt.run()
        profit[n_idx][min_diff_idx] = output['Profit Factor']
        raise

In [ ]:
class Teste1(Strategy):
                def init(self):
                    pass
                #    high = self.data.High
                #    self.array = self.I(fesx, high,scatter=False)
                def next(self):
                    if consec_n_down(self.data.High, 3, 0):
                        self.sell()
                    if consec_n_up(self.data.High, 3, 0):
                        self.buy()

bt = Backtest(df_fesx, Teste1,
            cash=10000, commission=.002,
            exclusive_orders=True)

output = bt.run()
bt.plot(superimpose=False)

In [ ]:
fig = go.Figure(data=[go.Surface(z=profit)])
fig.show()